<span style="color:red; font-size:16px; font-weight:bold">Instruction:</span><br>
Students must write their solutions exclusively in the sections marked with:<br>
<span style="color:red; font-size:15px; font-weight:bold"># CODE HERE !<br><br>
###########
</span>

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import sklearn

In [ ]:
# For helper functions only
# Write your helper functions here (if any)
# CODE HERE !
import os, sys, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from itertools import combinations
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

BASE = "./data"  # 改成你的实际数据根目录
RNG = np.random.default_rng(42)

def load_dataset(base_dir, period="2010"):
    sub = "2010_2015" if period == "2010" else "2020_2024"
    root = os.path.join(base_dir, sub)

    # 主+时间变化在同一对文件里
    pos_main_tc = np.load(os.path.join(root, "pos_features_main_timechange.npy"))
    neg_main_tc = np.load(os.path.join(root, "neg_features_main_timechange.npy"))
    X_main_tc = np.vstack([pos_main_tc, neg_main_tc])

    # 按列切片：FS-I 是前 18 列；FS-II 是第 19-90 列（索引 18:90）
    X_fsi  = X_main_tc[:, 0:18]
    X_fsii = X_main_tc[:, 18:90]

    # FS-III（历史活动，通常形状 (N,1)）
    pos_hist = np.load(os.path.join(root, "pos_features_historical.npy"))
    neg_hist = np.load(os.path.join(root, "neg_features_historical.npy"))
    X_fsiii = np.vstack([pos_hist, neg_hist])

    # FS-IV（极差）
    pos_mm = np.load(os.path.join(root, "pos_features_maxmin.npy"))
    neg_mm = np.load(os.path.join(root, "neg_features_maxmin.npy"))
    X_fsiv = np.vstack([pos_mm, neg_mm])

    # 标签：正样本=1 在前，负样本=0 在后
    y = np.hstack([
        np.ones(pos_main_tc.shape[0], dtype=int),
        np.zeros(neg_main_tc.shape[0], dtype=int)
    ])

    # 用 data_order 重排，确保与作业的观测顺序一致
    order = np.load(os.path.join(root, "data_order.npy")).astype(int)
    X_fsi  = X_fsi[order]
    X_fsii = X_fsii[order]
    X_fsiii= X_fsiii[order]
    X_fsiv = X_fsiv[order]
    y      = y[order]

    return {"features": {"FS-I": X_fsi, "FS-II": X_fsii, "FS-III": X_fsiii, "FS-IV": X_fsiv},
            "y": y}

def combine_features(features_dict, combo):
    """
    features_dict: {"FS-I":X1, ...}
    combo: 例如 ("FS-I",) 或 ("FS-I","FS-II") 等
    返回拼接后的 X
    """
    Xs = [features_dict[k] for k in combo]
    return np.hstack(Xs)

def all_feature_combos(keys=("FS-I","FS-II","FS-III","FS-IV")):
    # 4 个特征集的非空组合，共 15 种
    res = []
    for r in range(1, len(keys)+1):
        for c in combinations(keys, r):
            res.append(c)
    return res

def compute_tss(y_true, y_pred):
    """
    TSS = TP/(TP+FN) - FP/(FP+TN)
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    recall = tp / (tp + fn) if (tp+fn) > 0 else 0.0
    fpr    = fp / (fp + tn) if (fp+tn) > 0 else 0.0
    return recall - fpr, (tn, fp, fn, tp)
###########

### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Initialization Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 1</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Preprocess Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 3</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Feature Creation Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 2</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Cross Validation Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 5</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Training Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 2</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">TSS Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 2</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

In [ ]:
# __init__() function should initialize all your variables
# _____ 1 pt _____
class my_svm():
    def __init__(self, C=1.0, kernel="rbf", gamma="scale", n_splits=5, random_state=42):
        # CODE HERE !
        self.C = C
        self.kernel = kernel
        self.gamma = gamma
        self.n_splits = n_splits
        self.random_state = random_state

        # 运行期对象
        self.scaler = None
        self.model = None
        self.last_conf_mx = None  # (tn, fp, fn, tp)
        self.last_fold_scores = None  # 每折 TSS
        ###########
        #pass
#############################################################


    # preprocess() function:
    # _____ 1 pt _____
    #  1) normalizes the data, 
    # _____ 1 pt _____
    #  2) removes missing values
    # _____ 1 pt _____
    #  3) assign labels to target
    def preprocess(self, X, y):
        # CODE HERE !
        mask = np.isfinite(X).all(axis=1)
        X, y = X[mask], y[mask]
        self.scaler = StandardScaler()
        Xs = self.scaler.fit_transform(X)
        return Xs, y
        ###########
        #return 0
#############################################################

    # _____ 2 pt _____
    # feature_creation() function takes as input the features set label (e.g. FS-I, FS-II, FS-III, FS-IV)
    # and creates 2 D array of corresponding features 
    # for both positive and negative observations.
    # this array will be input to the svm model
    # For instance, if the input is FS-I, the output is a 2-d array with features corresponding to 
    # FS-I for both negative and positive class observations
    def feature_creation(self, features_dict, combo):
        # CODE HERE !
        return combine_features(features_dict, combo)
        ###########
        #return 0
#############################################################


    # cross_validation() function splits the data into train and test splits,
    # _____ 1 pt _____
    # Use k-fold with k=10
    # _____ 1 pt _____
    # the svm is trained on training set and tested on test set
    # _____ 1 pt _____
    # the output is the average accuracy across all train test splits.
    # _____ 2 pt _____ (Integration of the two functions)
    def cross_validation(self, X, y):
        # CODE HERE !
        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        fold_scores = []
        for tr_idx, te_idx in skf.split(X, y):
            Xtr, Xte = X[tr_idx], X[te_idx]
            ytr, yte = y[tr_idx], y[te_idx]
            scaler = StandardScaler()
            Xtr = scaler.fit_transform(Xtr)
            Xte = scaler.transform(Xte)

            clf = SVC(C=self.C, kernel=self.kernel, gamma=self.gamma, random_state=self.random_state)
            clf.fit(Xtr, ytr)
            yhat = clf.predict(Xte)
            tss_val, _ = compute_tss(yte, yhat)
            fold_scores.append(tss_val)

        self.last_fold_scores = np.array(fold_scores)
        return float(np.mean(fold_scores)), float(np.std(fold_scores)), self.last_fold_scores
        ###########
        # call training function
        # call tss function
        #return 0
#############################################################


    # _____ 2 pt _____
    #training() function trains a SVM classification model on input features and corresponding target
    def training(self, Xtr, ytr, Xte, yte):
        # CODE HERE !
        self.scaler = StandardScaler()
        Xtr = self.scaler.fit_transform(Xtr)
        Xte = self.scaler.transform(Xte)
        self.model = SVC(C=self.C, kernel=self.kernel, gamma=self.gamma, random_state=self.random_state)
        self.model.fit(Xtr, ytr)
        yhat = self.model.predict(Xte)
        tss_val, cm = compute_tss(yte, yhat)
        self.last_conf_mx = cm
        return tss_val, cm
        ###########
        #return 0
#############################################################


    # _____ 2 pt _____
    # tss() function computes the accuracy of predicted outputs (i.e target prediction on test set)
    # using the TSS measure given in the document
    def tss(self, y_true, y_pred):
        # CODE HERE !
        tss_val, cm = compute_tss(y_true, y_pred)
        self.last_conf_mx = cm
        return tss_val
        ###########
        #return 0


### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Feature Experiment Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 4</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

In [ ]:
# _____ 1 pt _____ (Runs all feature set combinations)
# _____ 1 pt _____ (Reports mean/std TSS)
# _____ 1 pt _____ (Produces confusion matrices)
# _____ 1 pt _____ (Generates bar chart of fold results)

# feature_experiment() function executes experiments with all 4 feature sets.
# svm is trained (and tested) on 2010 dataset with all 4 feature set combinations
# the output of this function is : 
#  1) TSS average scores (mean std) for k-fold validation printed out on console.
#  2) Confusion matrix for TP, FP, TN, FN. See assignment document 
#  3) A chart showing TSS scores for all folds of CV. 
#     This means that for each fold, compute the TSS score on test set for that fold, and plot it.
#     The number of folds will decide the number of points on this chart (i.e 10)
#
# Above 3 charts are produced for all feature combinations
#  4) The function prints the best performing feature set combination
def feature_experiment():
    # CODE HERE !
    """
    在 2010-2015 数据集上，遍历 15 种特征组合：
      - 打印每种组合的 5 折 TSS 均值/标准差
      - 画出每种组合的折内 TSS 柱状图
      - 为每种组合给出一次 holdout 的混淆矩阵（按 80/20 随机切分）
      - 打印最佳组合及其 TSS
    """
    # 你把这个路径改成你存放 *.npy 的根目录
    BASE = "./data"  # e.g. ".../GOES_data"
    data = load_dataset(BASE, period="2010")
    feats = data["features"]
    y = data["y"]

    keys = ("FS-I","FS-II","FS-III","FS-IV")
    combos = all_feature_combos(keys)

    best = {"combo": None, "mean": -1.0, "std": None}

    for combo in combos:
        X = combine_features(feats, combo)
        clf = my_svm(C=1.0, kernel="rbf", gamma="scale", n_splits=5, random_state=42)

        Xp, yp = clf.preprocess(X, y)
        mean_tss, std_tss, fold_scores = clf.kfold_cv(Xp, yp)
        print(f"[2010] Combo={combo} | TSS (mean±std) = {mean_tss:.4f} ± {std_tss:.4f}")

        # 折内 TSS 柱状图
        plt.figure()
        plt.bar(np.arange(len(fold_scores)), fold_scores)
        plt.title(f"TSS per fold (2010) - {combo}")
        plt.xlabel("Fold")
        plt.ylabel("TSS")
        plt.show()

        # 简单 holdout，画一次混淆矩阵
        n = len(yp)
        idx = np.arange(n)
        RNG.shuffle(idx)
        cut = int(n*0.8)
        tr, te = idx[:cut], idx[cut:]
        tss_hold, cm = clf.train_test(Xp[tr], yp[tr], Xp[te], yp[te])

        from sklearn.metrics import ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=np.array([[cm[0], cm[1]],[cm[2], cm[3]]]),
                                      display_labels=[0,1])
        fig, ax = plt.subplots()
        disp.plot(ax=ax)
        ax.set_title(f"Confusion Matrix (2010) - {combo} | TSS={tss_hold:.3f}")
        plt.show()

        if mean_tss > best["mean"]:
            best = {"combo": combo, "mean": mean_tss, "std": std_tss}

    print(f"\n== Best feature combo on 2010 == {best['combo']} | TSS={best['mean']:.4f} ± {best['std']:.4f}")
    return best
    ###########
    #return 0


### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Data Experiment Function</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 4</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

In [ ]:
# _____ 1 pt _____ (Runs on both datasets)
# _____ 1 pt _____ (Uses best feature set)
# _____ 1 pt _____ (Reports mean/std TSS)
# _____ 1 pt _____ (Produces confusion matrices + fold charts)

# data_experiment() function executes 2 experiments with 2 data sets.
# svm is trained (and tested) on both 2010 data and 2020 data
# the output of this function is : 
#  1) TSS average scores for k-fold validation printed out on console.
#  2) Confusion matrix for TP, FP, TN, FN. See assignment document 
#  3) A chart showing TSS scores for all folds of CV. 
#     This means that for each fold, compute the TSS score on test set for that fold, and plot it.
#     The number of folds will decide the number of points on this chart (i.e. 10)
# above 3 charts are produced for both datasets
# feature set for this experiment should be the 
# best performing feature set combination from feature_experiment()
def data_experiment():
    # CODE HERE !
    """
    用 feature_experiment() 找到的最佳特征组合，分别在 2010 和 2020 数据集上跑：
      - 打印 5 折 TSS 均值/标准差
      - 画折内 TSS 柱状图
      - 画一次 holdout 混淆矩阵
    """
    BASE = "./data"  # 改成你的 *.npy 根目录
    # 先拿到最佳组合
    best = feature_experiment()
    best_combo = best["combo"]

    for period, title in [("2010", "2010-2015"), ("2020", "2020-2024")]:
        data = load_dataset(BASE, period=period)
        feats, y = data["features"], data["y"]
        X = combine_features(feats, best_combo)

        clf = my_svm(C=1.0, kernel="rbf", gamma="scale", n_splits=5, random_state=42)
        Xp, yp = clf.preprocess(X, y)
        mean_tss, std_tss, fold_scores = clf.kfold_cv(Xp, yp)
        print(f"[{title}] BestCombo={best_combo} | TSS (mean±std) = {mean_tss:.4f} ± {std_tss:.4f}")

        # 折内 TSS 柱状图
        plt.figure()
        plt.bar(np.arange(len(fold_scores)), fold_scores)
        plt.title(f"TSS per fold ({title}) - {best_combo}")
        plt.xlabel("Fold")
        plt.ylabel("TSS")
        plt.show()

        # 一次 holdout 的混淆矩阵
        n = len(yp); idx = np.arange(n); RNG.shuffle(idx); cut = int(n*0.8)
        tr, te = idx[:cut], idx[cut:]
        tss_hold, cm = clf.train_test(Xp[tr], yp[tr], Xp[te], yp[te])

        from sklearn.metrics import ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=np.array([[cm[0], cm[1]],[cm[2], cm[3]]]),
                                      display_labels=[0,1])
        fig, ax = plt.subplots()
        disp.plot(ax=ax)
        ax.set_title(f"Confusion Matrix ({title}) - {best_combo} | TSS={tss_hold:.3f}")
        plt.show()
    ###########
    #return 0


### ────────────────────────────────────────────────────────────────
### <span style="color:blue; font-size:18px; font-weight:bold">Main Execution</span>
###### <span style="color:red; font-size:16px">**Points:** ___ / 2</span>
###### <span style="color:green; font-size:16px">**Comments:**</span>

In [ ]:
# _____ 1 pt _____ (Clear printed output (best feature set, dataset comparison))
# _____ 1 pt _____ (Visualizations labeled and interpretable)

# below should be your code to call the above classes and functions
# with various combinations of feature sets
# and both datasets

# CODE HERE !

###########

feature_experiment()
data_experiment()

<span style="color:blue; font-size:20px; font-weight:bold">Final Total Score</span><br>
<span style="color:red; font-size:18px">**____ / 25**</span>
</div>